# ES → BBF Migration Data Analysis (v4)

This notebook analyzes all data required for the ES to BBF Salesforce migration.

## Version 4 Changes
- **NEW**: OrderItem analysis for migration scope orders
- **NEW**: OrderItem data quality checks
- **NEW**: Product/Family breakdown analysis
- **NEW**: Proposed field mapping reference
- **FIX**: Added `Billing_Start_Date__c` to date cascade (Service_Start_Date__c → Billing_Start_Date__c → OSS bill_start_date)

## Version 3 Changes (inherited)
- Include OA (Accepted) orders in addition to CL (Closed)
- Work Orders enriched with data from `workorders.workorders` table

## Driving Principle
**Everything is driven from Active ES Orders that are ACTUALLY BILLING in OSS** - we migrate only the data needed to support truly active services.

## Filter Pipeline
1. Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
2. Project_Group__c NOT LIKE '%PA MARKET DECOM%'
3. Service_Order_Record_Type__c = 'Service Order Agreement' (excludes Work Orders)
4. OSS Actively Billing:
   - order_state_cd IN ('CL', 'OA')
   - bill_start_date <= today
   - bill_end_date IS NULL or > today
5. Has BBF BAN mapping

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from collections import defaultdict

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [2]:
# === CONFIGURATION ===

# ES (Source) Salesforce Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # 'login' for production

# OSS Database Credentials
OSS_HOST = "pg01.comlink.net"
OSS_PORT = "5432"
OSS_DB = "GLC"
OSS_USER = "oss_server"
OSS_PASSWORD = "3wU3uB28X?!r2?@ebrUg"

# Active Order Status Filter
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# Record Type Filter
VALID_RECORD_TYPE = "Service Order Agreement"

# PA Market Decom Exclusion
PA_DECOM_FILTER = "PA MARKET DECOM"

# OSS Order States that qualify as "actively billing"
ACTIVE_OSS_STATES = ["CL", "OA"]  # Closed and Accepted

# OSS Order States Reference
OSS_ORDER_STATES = {
    "CL": "Closed (Active/Billing)",
    "OA": "Accepted",
    "OS": "Submitted",
    "OC": "Created",
    "PN": "Pending",
    "CA": "Cancelled",
    "OR": "Rejected",
    "OV": "Validated (Disabled)",
}

# OSS Work Order Types Reference
WORKORDER_TYPES = {
    "IT": "Professional Services",
    "MR": "Maintenance/Repair",
    "OS": "Other Service",
    "VS": "Voice Service",
}

# OSS Work Order States Reference
WORKORDER_STATES = {
    "CA": "Cancelled",
    "CL": "Closed",
    "OA": "Accepted",
    "PN": "Pending",
}

# Output Configuration
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"es_bbf_migration_analysis_v4_{TIMESTAMP}.xlsx"

print("📋 Configuration loaded")
print(f"   Active Statuses: {ACTIVE_STATUSES}")
print(f"   Record Type: {VALID_RECORD_TYPE}")
print(f"   Excluding: Project_Group__c LIKE '%{PA_DECOM_FILTER}%'")
print(f"   OSS Active States: {ACTIVE_OSS_STATES}")
print(f"   Output: {OUTPUT_FILE}")

📋 Configuration loaded
   Active Statuses: ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
   Record Type: Service Order Agreement
   Excluding: Project_Group__c LIKE '%PA MARKET DECOM%'
   OSS Active States: ['CL', 'OA']
   Output: es_bbf_migration_analysis_v4_20260107_212249.xlsx


In [3]:
# === CONNECT TO ES SALESFORCE ===

print("=" * 80)
print("CONNECTING TO ES SALESFORCE")
print("=" * 80)

print("\n🔌 Connecting to ES...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

CONNECTING TO ES SALESFORCE

🔌 Connecting to ES...
✅ Connected to ES: everstream.my.salesforce.com


In [4]:
# === CONNECT TO OSS DATABASE ===

print("\n" + "=" * 80)
print("CONNECTING TO OSS DATABASE")
print("=" * 80)

print("\n🔌 Connecting to OSS...")
oss_conn = psycopg2.connect(
    dbname=OSS_DB,
    user=OSS_USER,
    password=OSS_PASSWORD,
    host=OSS_HOST,
    port=OSS_PORT,
)
print(f"✅ Connected to OSS: {OSS_HOST}/{OSS_DB}")


CONNECTING TO OSS DATABASE

🔌 Connecting to OSS...
✅ Connected to OSS: pg01.comlink.net/GLC


In [5]:
# === STEP 1: QUERY ALL ACTIVE ORDERS ===

print("\n" + "=" * 80)
print("STEP 1: QUERYING ACTIVE ORDERS")
print("=" * 80)

status_filter = "','".join(ACTIVE_STATUSES)

orders_query = f"""
SELECT 
    Id, 
    Name,
    Service_ID__c,
    Status,
    AccountId,
    Account.Name,
    Billing_Invoice__c,
    Address_A__c,
    Address_Z__c,
    Node__c,
    OpportunityId,
    Service_Start_Date__c,
    Billing_Start_Date__c,
    Service_End_Date__c,
    Service_Provided__c,
    SOF_MRC__c,
    OSS_Order__c,
    OSS_Service_ID__c,
    Vendor_Circuit_ID__c,
    Primary_Product_Family__c,
    Primary_Product_Name__c,
    Project_Group__c,
    Service_Order_Record_Type__c,
    CreatedDate,
    LastModifiedDate
FROM Order
WHERE Status IN ('{status_filter}')
ORDER BY Service_ID__c
"""

print("Querying all orders with active statuses...")
result = es_sf.query_all(orders_query)
orders_raw = result["records"]

# Flatten results
all_orders = []
for order in orders_raw:
    all_orders.append(
        {
            "Id": order["Id"],
            "Name": order.get("Name"),
            "Service_ID__c": order.get("Service_ID__c"),
            "Status": order["Status"],
            "AccountId": order.get("AccountId"),
            "Account_Name": order["Account"]["Name"] if order.get("Account") else None,
            "Billing_Invoice__c": order.get("Billing_Invoice__c"),
            "Address_A__c": order.get("Address_A__c"),
            "Address_Z__c": order.get("Address_Z__c"),
            "Node__c": order.get("Node__c"),
            "OpportunityId": order.get("OpportunityId"),
            "Service_Start_Date__c": order.get("Service_Start_Date__c"),
            "Billing_Start_Date__c": order.get("Billing_Start_Date__c"),
            "Service_End_Date__c": order.get("Service_End_Date__c"),
            "Service_Provided__c": order.get("Service_Provided__c"),
            "SOF_MRC__c": order.get("SOF_MRC__c"),
            "OSS_Order__c": order.get("OSS_Order__c"),
            "OSS_Service_ID__c": order.get("OSS_Service_ID__c"),
            "Vendor_Circuit_ID__c": order.get("Vendor_Circuit_ID__c"),
            "Primary_Product_Family__c": order.get("Primary_Product_Family__c"),
            "Primary_Product_Name__c": order.get("Primary_Product_Name__c"),
            "Project_Group__c": order.get("Project_Group__c"),
            "Service_Order_Record_Type__c": order.get("Service_Order_Record_Type__c"),
            "CreatedDate": order.get("CreatedDate"),
            "LastModifiedDate": order.get("LastModifiedDate"),
        }
    )

all_orders_df = pd.DataFrame(all_orders)
print(f"\n✅ Total orders with active status: {len(all_orders_df):,}")

# Show record type breakdown
print(f"\n📊 Record Type Breakdown:")
print(all_orders_df["Service_Order_Record_Type__c"].value_counts(dropna=False))


STEP 1: QUERYING ACTIVE ORDERS
Querying all orders with active statuses...

✅ Total orders with active status: 17,972

📊 Record Type Breakdown:
Service_Order_Record_Type__c
Service Order Agreement    14058
Work Order                  3914
Name: count, dtype: int64


In [6]:
# === STEP 2: FILTER - PA MARKET DECOM ===

print("\n" + "=" * 80)
print("STEP 2: FILTER - PA MARKET DECOM")
print("=" * 80)

# Separate PA MARKET DECOM orders
pa_decom_mask = (
    all_orders_df["Project_Group__c"]
    .fillna("")
    .str.contains(PA_DECOM_FILTER, case=False)
)
excluded_pa_decom_df = all_orders_df[pa_decom_mask].copy()
orders_after_pa_filter = all_orders_df[~pa_decom_mask].copy()

print(f"\n📊 PA MARKET DECOM Filter:")
print(f"   Before: {len(all_orders_df):,}")
print(f"   ❌ Excluded (PA MARKET DECOM): {len(excluded_pa_decom_df):,}")
print(f"   ✅ Remaining: {len(orders_after_pa_filter):,}")


STEP 2: FILTER - PA MARKET DECOM

📊 PA MARKET DECOM Filter:
   Before: 17,972
   ❌ Excluded (PA MARKET DECOM): 887
   ✅ Remaining: 17,085


In [7]:
# === STEP 3: FILTER - RECORD TYPE (Service Order Agreement only) ===

print("\n" + "=" * 80)
print("STEP 3: FILTER - RECORD TYPE")
print("=" * 80)

# Keep only Service Order Agreement
record_type_mask = (
    orders_after_pa_filter["Service_Order_Record_Type__c"] == VALID_RECORD_TYPE
)
excluded_work_orders_df = orders_after_pa_filter[~record_type_mask].copy()
orders_after_rt_filter = orders_after_pa_filter[record_type_mask].copy()

print(f"\n📊 Record Type Filter:")
print(f"   Before: {len(orders_after_pa_filter):,}")
print(f"   ❌ Excluded (Not '{VALID_RECORD_TYPE}'): {len(excluded_work_orders_df):,}")
print(f"   ✅ Remaining: {len(orders_after_rt_filter):,}")

# Show breakdown of what was excluded
if len(excluded_work_orders_df) > 0:
    print(f"\n   Excluded Record Types:")
    print(
        excluded_work_orders_df["Service_Order_Record_Type__c"].value_counts(
            dropna=False
        )
    )


STEP 3: FILTER - RECORD TYPE

📊 Record Type Filter:
   Before: 17,085
   ❌ Excluded (Not 'Service Order Agreement'): 3,477
   ✅ Remaining: 13,608

   Excluded Record Types:
Service_Order_Record_Type__c
Work Order    3477
Name: count, dtype: int64


In [8]:
# === STEP 3B: ENRICH WORK ORDERS WITH OSS DATA ===

print("\n" + "=" * 80)
print("STEP 3B: ENRICHING WORK ORDERS WITH OSS DATA")
print("=" * 80)

if len(excluded_work_orders_df) > 0:
    # Get work orders with OSS_Order__c (maps to workorder_id)
    has_wo_oss = excluded_work_orders_df["OSS_Order__c"].notna() & (
        excluded_work_orders_df["OSS_Order__c"] != ""
    )
    work_orders_with_oss = excluded_work_orders_df[has_wo_oss].copy()

    print(f"\n   Work Orders with OSS_Order__c: {len(work_orders_with_oss):,}")
    print(f"   Work Orders without OSS_Order__c: {(~has_wo_oss).sum():,}")

    if len(work_orders_with_oss) > 0:
        workorder_ids = (
            work_orders_with_oss["OSS_Order__c"].dropna().astype(int).unique().tolist()
        )
        print(f"   Unique workorder IDs to query: {len(workorder_ids):,}")

        # Query workorders.workorders
        chunk_size = 5000
        oss_workorders = []

        for i in range(0, len(workorder_ids), chunk_size):
            chunk = workorder_ids[i : i + chunk_size]
            ids_str = ",".join(str(x) for x in chunk)

            wo_query = f"""
            SELECT 
                workorder_id,
                order_nm,
                order_id,
                workorder_type_cd,
                workorder_state_cd,
                description,
                start_date,
                end_date,
                disabled
            FROM workorders.workorders
            WHERE workorder_id IN ({ids_str})
              AND disabled = 'infinity'
            """

            with oss_conn.cursor(cursor_factory=RealDictCursor) as cur:
                cur.execute(wo_query)
                rows = cur.fetchall()
                oss_workorders.extend([dict(row) for row in rows])

            print(f"   Chunk {i//chunk_size + 1}: Retrieved {len(rows)} workorders")

        if len(oss_workorders) > 0:
            oss_wo_df = pd.DataFrame(oss_workorders)

            # Convert date columns - strip timezone for Excel compatibility
            for date_col in ["start_date", "end_date"]:
                if date_col in oss_wo_df.columns:
                    oss_wo_df[date_col] = pd.to_datetime(
                        oss_wo_df[date_col], utc=True
                    ).dt.tz_localize(None)

            # Add descriptions
            oss_wo_df["workorder_type_desc"] = oss_wo_df["workorder_type_cd"].map(
                WORKORDER_TYPES
            )
            oss_wo_df["workorder_state_desc"] = oss_wo_df["workorder_state_cd"].map(
                WORKORDER_STATES
            )

            # Merge with excluded work orders
            excluded_work_orders_df["OSS_Order_ID"] = pd.to_numeric(
                excluded_work_orders_df["OSS_Order__c"], errors="coerce"
            )
            excluded_work_orders_df = excluded_work_orders_df.merge(
                oss_wo_df, left_on="OSS_Order_ID", right_on="workorder_id", how="left"
            )

            print(
                f"\n✅ Enriched {oss_wo_df['workorder_id'].notna().sum():,} work orders with OSS data"
            )
        else:
            print("\n⚠️ No active workorders found in OSS")
    else:
        print("\n⚠️ No work orders have OSS_Order__c populated")
else:
    print("\n⚠️ No work orders to enrich")


STEP 3B: ENRICHING WORK ORDERS WITH OSS DATA

   Work Orders with OSS_Order__c: 156
   Work Orders without OSS_Order__c: 3,321
   Unique workorder IDs to query: 156
   Chunk 1: Retrieved 84 workorders

✅ Enriched 84 work orders with OSS data


In [9]:
# === STEP 4: CHECK OSS ACTIVELY BILLING STATUS ===

print("\n" + "=" * 80)
print("STEP 4: CHECKING OSS ACTIVELY BILLING STATUS")
print("=" * 80)

# Identify orders with OSS_Order__c
has_oss_id = orders_after_rt_filter["OSS_Order__c"].notna() & (
    orders_after_rt_filter["OSS_Order__c"] != ""
)
orders_with_oss = orders_after_rt_filter[has_oss_id].copy()
orders_without_oss = orders_after_rt_filter[~has_oss_id].copy()

print(f"\n📊 OSS_Order__c Population:")
print(f"   With OSS_Order__c: {len(orders_with_oss):,}")
print(f"   Without OSS_Order__c: {len(orders_without_oss):,}")

# Query OSS for orders with OSS_Order__c
if len(orders_with_oss) > 0:
    oss_order_ids = (
        orders_with_oss["OSS_Order__c"].dropna().astype(int).unique().tolist()
    )
    print(f"\n   Unique OSS Order IDs to query: {len(oss_order_ids):,}")

    # Query in chunks
    chunk_size = 5000
    oss_orders = []

    for i in range(0, len(oss_order_ids), chunk_size):
        chunk = oss_order_ids[i : i + chunk_size]
        ids_str = ",".join(str(x) for x in chunk)

        oss_query = f"""
        SELECT 
            order_id,
            order_state_cd,
            order_type_cd,
            bill_start_date,
            bill_end_date,
            circuit_active_date,
            account_id,
            service_id
        FROM om.orders
        WHERE order_id IN ({ids_str})
        """

        with oss_conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(oss_query)
            rows = cur.fetchall()
            oss_orders.extend([dict(row) for row in rows])

        print(f"   Chunk {i//chunk_size + 1}: Retrieved {len(rows)} orders")

    oss_orders_df = pd.DataFrame(oss_orders)
    print(f"\n✅ Total OSS orders retrieved: {len(oss_orders_df):,}")

    # Check for OSS IDs not found
    found_ids = (
        set(oss_orders_df["order_id"].tolist()) if len(oss_orders_df) > 0 else set()
    )
    not_found_ids = set(oss_order_ids) - found_ids
    print(f"   OSS Order IDs not found in OSS: {len(not_found_ids):,}")
else:
    oss_orders_df = pd.DataFrame()
    not_found_ids = set()


STEP 4: CHECKING OSS ACTIVELY BILLING STATUS

📊 OSS_Order__c Population:
   With OSS_Order__c: 11,525
   Without OSS_Order__c: 2,083

   Unique OSS Order IDs to query: 11,518
   Chunk 1: Retrieved 5000 orders
   Chunk 2: Retrieved 4999 orders
   Chunk 3: Retrieved 1518 orders

✅ Total OSS orders retrieved: 11,517
   OSS Order IDs not found in OSS: 1


In [10]:
# === STEP 5: DETERMINE ACTIVELY BILLING STATUS ===

print("\n" + "=" * 80)
print("STEP 5: DETERMINING ACTIVELY BILLING STATUS")
print("=" * 80)

today = pd.Timestamp.now().normalize()
print(f"\nToday's date for comparison: {today.date()}")
print(f"Active OSS States: {ACTIVE_OSS_STATES}")

# Merge SF orders with OSS data
if len(orders_with_oss) > 0:
    orders_with_oss["OSS_Order_ID"] = orders_with_oss["OSS_Order__c"].astype(int)

if len(oss_orders_df) > 0:
    # Convert date columns
    oss_orders_df["bill_start_date"] = pd.to_datetime(
        oss_orders_df["bill_start_date"], utc=True
    ).dt.tz_localize(None)
    oss_orders_df["bill_end_date"] = pd.to_datetime(
        oss_orders_df["bill_end_date"], utc=True
    ).dt.tz_localize(None)

    # Merge
    merged_df = orders_with_oss.merge(
        oss_orders_df, left_on="OSS_Order_ID", right_on="order_id", how="left"
    )
else:
    merged_df = orders_with_oss.copy()
    merged_df["order_id"] = None
    merged_df["order_state_cd"] = None
    merged_df["bill_start_date"] = None
    merged_df["bill_end_date"] = None


# Determine actively billing status
def get_billing_status(row):
    if pd.isna(row.get("order_id")):
        return "No OSS Match"

    state = row["order_state_cd"].strip() if row.get("order_state_cd") else None
    bill_start = row.get("bill_start_date")
    bill_end = row.get("bill_end_date")

    # Check if state is in allowed active states (CL or OA)
    if state not in ACTIVE_OSS_STATES:
        return f"Not Active State ({state})"

    if pd.isna(bill_start):
        return f"{state} - No Bill Start Date"

    if bill_start > today:
        return f"{state} - Bill Start Future"

    if pd.notna(bill_end) and bill_end <= today:
        return f"{state} - Bill End Passed"

    return "ACTIVELY BILLING"


merged_df["OSS_Billing_Status"] = merged_df.apply(get_billing_status, axis=1)

# Add state description
merged_df["OSS_State_Desc"] = merged_df["order_state_cd"].map(OSS_ORDER_STATES)

# Also handle orders without OSS link
orders_without_oss["OSS_Billing_Status"] = "No OSS_Order__c in SF"
orders_without_oss["order_state_cd"] = None
orders_without_oss["OSS_State_Desc"] = None
orders_without_oss["bill_start_date"] = None
orders_without_oss["bill_end_date"] = None

# Summary
print("\n📊 OSS Billing Status Distribution (orders with OSS_Order__c):")
if len(merged_df) > 0:
    billing_status_counts = merged_df["OSS_Billing_Status"].value_counts()
    for status, count in billing_status_counts.items():
        pct = 100 * count / len(merged_df)
        print(f"   {status}: {count:,} ({pct:.1f}%)")


STEP 5: DETERMINING ACTIVELY BILLING STATUS

Today's date for comparison: 2026-01-07
Active OSS States: ['CL', 'OA']

📊 OSS Billing Status Distribution (orders with OSS_Order__c):
   ACTIVELY BILLING: 11,489 (99.7%)
   CL - Bill End Passed: 23 (0.2%)
   Not Active State (CA): 6 (0.1%)
   CL - Bill Start Future: 6 (0.1%)
   No OSS Match: 1 (0.0%)


In [11]:
# === STEP 6: FILTER - ACTIVELY BILLING ONLY ===

print("\n" + "=" * 80)
print("STEP 6: FILTER - ACTIVELY BILLING ONLY")
print("=" * 80)

# Get common columns
common_cols = [col for col in orders_after_rt_filter.columns]
oss_cols = [
    "order_state_cd",
    "OSS_State_Desc",
    "bill_start_date",
    "bill_end_date",
    "OSS_Billing_Status",
]

# Ensure all columns exist
for col in oss_cols:
    if col not in merged_df.columns:
        merged_df[col] = None
    if col not in orders_without_oss.columns:
        orders_without_oss[col] = None

all_cols = common_cols + oss_cols

# Combine orders with and without OSS
all_checked_orders = pd.concat(
    [
        merged_df[[c for c in all_cols if c in merged_df.columns]],
        orders_without_oss[[c for c in all_cols if c in orders_without_oss.columns]],
    ],
    ignore_index=True,
)

# Split into actively billing vs not
actively_billing_mask = all_checked_orders["OSS_Billing_Status"] == "ACTIVELY BILLING"
orders_actively_billing = all_checked_orders[actively_billing_mask].copy()
excluded_not_billing_df = all_checked_orders[~actively_billing_mask].copy()

print(f"\n📊 Actively Billing Filter:")
print(f"   Before: {len(all_checked_orders):,}")
print(f"   ❌ Excluded (Not Actively Billing): {len(excluded_not_billing_df):,}")
print(f"   ✅ Confirmed Actively Billing: {len(orders_actively_billing):,}")

# Show breakdown of excluded
if len(excluded_not_billing_df) > 0:
    print(f"\n   Breakdown of excluded:")
    for status, count in (
        excluded_not_billing_df["OSS_Billing_Status"].value_counts().items()
    ):
        print(f"      {status}: {count:,}")

# Set this as our active orders for the rest of the pipeline
active_orders_df = orders_actively_billing.copy()


STEP 6: FILTER - ACTIVELY BILLING ONLY

📊 Actively Billing Filter:
   Before: 13,608
   ❌ Excluded (Not Actively Billing): 2,119
   ✅ Confirmed Actively Billing: 11,489

   Breakdown of excluded:
      No OSS_Order__c in SF: 2,083
      CL - Bill End Passed: 23
      Not Active State (CA): 6
      CL - Bill Start Future: 6
      No OSS Match: 1


C:\Users\vjero\AppData\Local\Temp\ipykernel_41320\1019780369.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_checked_orders = pd.concat(
C:\Users\vjero\AppData\Local\Temp\ipykernel_41320\1019780369.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_checked_orders = pd.concat(


In [12]:
# === STEP 7: GET NEW BBF BANS ===

print("\n" + "=" * 80)
print("STEP 7: QUERYING NEW BBF BANS")
print("=" * 80)

bbf_ban_query = """
SELECT 
    Id,
    Name,
    Account__c,
    Account__r.Name,
    Legacy_ES_Id__c,
    BBF_Ban__c,
    Billing_Address_1__c,
    Billing_City__c,
    Billing_State__c,
    Billing_ZIP__c,
    Payment_Terms__c,
    Active_Billing__c
FROM Billing_Invoice__c
WHERE BBF_Ban__c = true
  AND Legacy_ES_Id__c != null
"""

print("Querying new BBF BANs (BBF_Ban__c = true, Legacy_ES_Id__c populated)...")
result = es_sf.query_all(bbf_ban_query)
bbf_bans_df = pd.DataFrame(result["records"])

if len(bbf_bans_df) > 0:
    # Flatten Account name
    if "Account__r" in bbf_bans_df.columns:
        bbf_bans_df["Account_Name"] = bbf_bans_df["Account__r"].apply(
            lambda x: x["Name"] if x else None
        )

    # Clean up
    bbf_bans_df = bbf_bans_df.drop(
        columns=["attributes", "Account__r"], errors="ignore"
    )

    print(f"\n✅ Found {len(bbf_bans_df):,} new BBF BANs")

    # Create lookup from legacy BAN ID to new BBF BAN
    legacy_to_bbf_ban = {}
    for _, ban in bbf_bans_df.iterrows():
        legacy_id = ban.get("Legacy_ES_Id__c")
        if legacy_id:
            legacy_to_bbf_ban[legacy_id] = {
                "Id": ban["Id"],
                "Name": ban["Name"],
                "Account__c": ban.get("Account__c"),
                "Account_Name": ban.get("Account_Name"),
            }
    print(f"   Legacy BAN to BBF BAN mappings: {len(legacy_to_bbf_ban):,}")
else:
    legacy_to_bbf_ban = {}
    print("⚠️ No BBF BANs found")


STEP 7: QUERYING NEW BBF BANS
Querying new BBF BANs (BBF_Ban__c = true, Legacy_ES_Id__c populated)...

✅ Found 2,505 new BBF BANs
   Legacy BAN to BBF BAN mappings: 2,505


In [13]:
# === STEP 8: MAP ORDERS TO BBF BANS ===

print("\n" + "=" * 80)
print("STEP 8: MAPPING ORDERS TO BBF BANS")
print("=" * 80)

# Track which orders can be mapped
orders_with_bbf_ban = []
orders_missing_bbf_ban = []
orders_missing_any_ban = []

for _, order in active_orders_df.iterrows():
    legacy_ban_id = order.get("Billing_Invoice__c")
    order_dict = order.to_dict()

    if not legacy_ban_id:
        # Order has no BAN at all
        orders_missing_any_ban.append(order_dict)
    elif legacy_ban_id in legacy_to_bbf_ban:
        # Order can be mapped to new BBF BAN
        bbf_ban = legacy_to_bbf_ban[legacy_ban_id]
        order_dict["New_BBF_BAN_Id"] = bbf_ban["Id"]
        order_dict["New_BBF_BAN_Name"] = bbf_ban["Name"]
        order_dict["New_BBF_BAN_Account__c"] = bbf_ban["Account__c"]
        order_dict["New_BBF_BAN_Account_Name"] = bbf_ban["Account_Name"]
        orders_with_bbf_ban.append(order_dict)
    else:
        # Order has legacy BAN but no BBF BAN mapping
        orders_missing_bbf_ban.append(order_dict)

orders_ready_df = pd.DataFrame(orders_with_bbf_ban)
orders_no_bbf_ban_df = pd.DataFrame(orders_missing_bbf_ban)
orders_no_ban_df = pd.DataFrame(orders_missing_any_ban)

print(f"\n📊 BAN Mapping Results:")
print(f"   ✅ Ready to migrate (has BBF BAN): {len(orders_ready_df):,}")
print(f"   ⚠️ Missing BBF BAN mapping: {len(orders_no_bbf_ban_df):,}")
print(f"   ❌ Missing ANY BAN: {len(orders_no_ban_df):,}")


STEP 8: MAPPING ORDERS TO BBF BANS

📊 BAN Mapping Results:
   ✅ Ready to migrate (has BBF BAN): 11,475
   ⚠️ Missing BBF BAN mapping: 12
   ❌ Missing ANY BAN: 2


In [14]:
# === STEP 9: IDENTIFY ACCOUNTS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 9: IDENTIFYING ACCOUNTS TO MIGRATE")
print("=" * 80)

if len(orders_ready_df) > 0:
    unique_account_ids = (
        orders_ready_df["New_BBF_BAN_Account__c"].dropna().unique().tolist()
    )
    print(f"\nFound {len(unique_account_ids):,} unique Accounts from new BBF BANs")

    if unique_account_ids:
        chunk_size = 150
        all_accounts = []

        for i in range(0, len(unique_account_ids), chunk_size):
            chunk = unique_account_ids[i : i + chunk_size]
            ids_str = "','".join(chunk)

            account_query = f"""
            SELECT Id, Name, Type, Industry, 
                   BillingStreet, BillingCity, BillingState, BillingPostalCode, BillingCountry,
                   Phone, Website, BBF_New_Id__c
            FROM Account
            WHERE Id IN ('{ids_str}')
            """
            result = es_sf.query_all(account_query)
            all_accounts.extend(result["records"])

        accounts_df = pd.DataFrame(all_accounts)
        if len(accounts_df) > 0:
            accounts_df = accounts_df.drop(columns=["attributes"], errors="ignore")
        print(f"\n✅ Accounts to migrate: {len(accounts_df):,}")
    else:
        accounts_df = pd.DataFrame()
else:
    accounts_df = pd.DataFrame()
    print("⚠️ No orders ready to migrate, skipping Account query")


STEP 9: IDENTIFYING ACCOUNTS TO MIGRATE

Found 2,224 unique Accounts from new BBF BANs

✅ Accounts to migrate: 2,224


In [15]:
# === STEP 10: IDENTIFY CONTACTS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 10: IDENTIFYING CONTACTS TO MIGRATE")
print("=" * 80)

if len(accounts_df) > 0:
    account_ids = accounts_df["Id"].tolist()
    chunk_size = 150
    all_contacts = []

    for i in range(0, len(account_ids), chunk_size):
        chunk = account_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        contact_query = f"""
        SELECT Id, AccountId, FirstName, LastName, Email, Phone, Title,
               MailingStreet, MailingCity, MailingState, MailingPostalCode,
               BBF_New_Id__c
        FROM Contact
        WHERE AccountId IN ('{ids_str}')
        """
        result = es_sf.query_all(contact_query)
        all_contacts.extend(result["records"])

    contacts_df = pd.DataFrame(all_contacts)
    if len(contacts_df) > 0:
        contacts_df = contacts_df.drop(columns=["attributes"], errors="ignore")
    print(f"\n✅ Contacts to migrate: {len(contacts_df):,}")
else:
    contacts_df = pd.DataFrame()
    print("⚠️ No accounts to migrate, skipping Contact query")


STEP 10: IDENTIFYING CONTACTS TO MIGRATE

✅ Contacts to migrate: 15,576


In [16]:
# === STEP 11: IDENTIFY LOCATIONS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 11: IDENTIFYING LOCATIONS TO MIGRATE")
print("=" * 80)

if len(orders_ready_df) > 0:
    address_a_ids = orders_ready_df["Address_A__c"].dropna().unique().tolist()
    address_z_ids = orders_ready_df["Address_Z__c"].dropna().unique().tolist()
    all_address_ids = list(set(address_a_ids + address_z_ids))

    print(f"\nUnique locations referenced by orders:")
    print(f"   Address_A: {len(address_a_ids):,}")
    print(f"   Address_Z: {len(address_z_ids):,}")
    print(f"   Combined unique: {len(all_address_ids):,}")

    if all_address_ids:
        chunk_size = 150
        all_locations = []

        for i in range(0, len(all_address_ids), chunk_size):
            chunk = all_address_ids[i : i + chunk_size]
            ids_str = "','".join(chunk)

            location_query = f"""
            SELECT Id, Name, Address__c, City__c, State__c, County__c, Zip__c,
                   Complete_Address__c, CLLI__c, Building_Status__c, On_Net__c,
                   BBF_New_Id__c
            FROM Address__c
            WHERE Id IN ('{ids_str}')
            """
            result = es_sf.query_all(location_query)
            all_locations.extend(result["records"])

        locations_df = pd.DataFrame(all_locations)
        if len(locations_df) > 0:
            locations_df = locations_df.drop(columns=["attributes"], errors="ignore")
        print(f"\n✅ Locations to migrate: {len(locations_df):,}")
    else:
        locations_df = pd.DataFrame()
else:
    locations_df = pd.DataFrame()
    all_address_ids = []
    print("⚠️ No orders ready to migrate, skipping Location query")


STEP 11: IDENTIFYING LOCATIONS TO MIGRATE

Unique locations referenced by orders:
   Address_A: 723
   Address_Z: 9,818
   Combined unique: 10,175

✅ Locations to migrate: 10,175


In [17]:
# === STEP 12: IDENTIFY OFF_NET RECORDS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 12: IDENTIFYING OFF_NET RECORDS TO MIGRATE")
print("=" * 80)

if len(all_address_ids) > 0:
    chunk_size = 100
    all_offnet = []

    for i in range(0, len(all_address_ids), chunk_size):
        chunk = all_address_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        offnet_query = f"""
        SELECT Id, Name, 
               Location_1__c, Location_1_Address__c,
               Location_2__c, Location_2_Address__c,
               Off_Net_Vendor__c, Vendor_Name__c,
               Vendor_circuit_Id__c, Internal_Circuit_Id__c,
               Cost_MRC__c, Cost_NRC__c, Invoice_MRC__c,
               LEC_Order_Status__c, Off_Net_Type__c,
               Bandwidth__c, Circuit_Type__c, Term__c,
               Term_Agreement_Start_Date__c, Term_Agreement_End_Date__c,
               Vendor_Bill_Start_Date__c, Vendor_Bill_Stop_Date__c,
               SOF1__c
        FROM Off_Net__c
        WHERE Location_1__c IN ('{ids_str}')
           OR Location_2__c IN ('{ids_str}')
        """
        result = es_sf.query_all(offnet_query)
        all_offnet.extend(result["records"])

    offnet_df = pd.DataFrame(all_offnet)
    if len(offnet_df) > 0:
        offnet_df = offnet_df.drop(columns=["attributes"], errors="ignore")
        offnet_df = offnet_df.drop_duplicates(subset=["Id"])
    print(f"\n✅ Off_Net records to migrate: {len(offnet_df):,}")
else:
    offnet_df = pd.DataFrame()
    print("⚠️ No locations to migrate, skipping Off_Net query")


STEP 12: IDENTIFYING OFF_NET RECORDS TO MIGRATE

✅ Off_Net records to migrate: 2,157


In [18]:
# === STEP 13: DATA QUALITY ANALYSIS ===

print("\n" + "=" * 80)
print("STEP 13: DATA QUALITY ANALYSIS")
print("=" * 80)

data_quality_issues = []

if len(orders_ready_df) > 0:
    total_ready = len(orders_ready_df)

    # Check for missing Address_A__c
    missing_addr_a = orders_ready_df["Address_A__c"].isna().sum()
    if missing_addr_a > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Address_A__c",
                "Count": missing_addr_a,
                "Percentage": f"{missing_addr_a/total_ready*100:.1f}%",
                "Severity": "HIGH",
                "Impact": "Cannot set A_Location__c on BBF Service__c",
            }
        )

    # Check for missing Node__c
    missing_node = orders_ready_df["Node__c"].isna().sum()
    if missing_node > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Node__c",
                "Count": missing_node,
                "Percentage": f"{missing_node/total_ready*100:.1f}%",
                "Severity": "LOW",
                "Impact": "Can fix post-migration - A_Node__c/Z_Node__c optional",
            }
        )

    # Check Billing/Service Date cascade: Service_Start_Date__c → Billing_Start_Date__c → OSS bill_start_date
    sf_svc_start_null = orders_ready_df["Service_Start_Date__c"].isna()
    sf_bill_start_null = orders_ready_df["Billing_Start_Date__c"].isna()
    oss_start_exists = orders_ready_df["bill_start_date"].notna()

    # Count coverage at each level
    has_svc_start = (~sf_svc_start_null).sum()
    has_bill_start_only = (sf_svc_start_null & ~sf_bill_start_null).sum()
    needs_oss = (sf_svc_start_null & sf_bill_start_null).sum()
    oss_covers = (sf_svc_start_null & sf_bill_start_null & oss_start_exists).sum()

    print(f"\n📊 Active Date Coverage (for Service__c.Active_Date__c):")
    print(
        f"   ✅ Has Service_Start_Date__c:           {has_svc_start:,} ({100*has_svc_start/total_ready:.1f}%)"
    )
    print(
        f"   ✅ Has Billing_Start_Date__c (fallback): {has_bill_start_only:,} ({100*has_bill_start_only/total_ready:.1f}%)"
    )
    print(
        f"   ⚠️ Needs OSS bill_start_date:           {needs_oss:,} ({100*needs_oss/total_ready:.1f}%)"
    )
    if needs_oss > 0:
        print(f"      └─ OSS covers: {oss_covers:,} of {needs_oss:,}")

    if sf_svc_start_null.sum() > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Service_Start_Date__c",
                "Count": sf_svc_start_null.sum(),
                "Percentage": f"{sf_svc_start_null.sum()/total_ready*100:.1f}%",
                "Severity": "LOW",
                "Impact": f"Billing_Start_Date__c covers {has_bill_start_only:,}, OSS covers {oss_covers:,} more",
            }
        )

data_quality_df = pd.DataFrame(data_quality_issues)

if len(data_quality_issues) > 0:
    print("\n📊 Data Quality Issues:")
    for issue in data_quality_issues:
        print(
            f"   [{issue['Severity']}] {issue['Issue']}: {issue['Count']} ({issue['Percentage']})"
        )
        print(f"         → {issue['Impact']}")
else:
    print("\n✅ No significant data quality issues found")


STEP 13: DATA QUALITY ANALYSIS

📊 Active Date Coverage (for Service__c.Active_Date__c):
   ✅ Has Service_Start_Date__c:           7,050 (61.4%)
   ✅ Has Billing_Start_Date__c (fallback): 4,425 (38.6%)
   ⚠️ Needs OSS bill_start_date:           0 (0.0%)

📊 Data Quality Issues:
   [HIGH] Orders missing Address_A__c: 2 (0.0%)
         → Cannot set A_Location__c on BBF Service__c
   [LOW] Orders missing Node__c: 9784 (85.3%)
         → Can fix post-migration - A_Node__c/Z_Node__c optional
   [LOW] Orders missing Service_Start_Date__c: 4425 (38.6%)
         → Billing_Start_Date__c covers 4,425, OSS covers 0 more


In [19]:
# === STEP 13B: QUERY ORDER ITEMS FOR MIGRATION SCOPE ===

print("\n" + "=" * 80)
print("STEP 13B: QUERYING ORDER ITEMS")
print("=" * 80)

if len(orders_ready_df) > 0:
    # Get all Order IDs from orders ready to migrate
    order_ids = orders_ready_df["Id"].unique().tolist()
    print(f"\nQuerying OrderItems for {len(order_ids):,} orders...")

    # Query in chunks
    chunk_size = 150
    all_order_items = []

    for i in range(0, len(order_ids), chunk_size):
        chunk = order_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        orderitem_query = f"""
        SELECT 
            Id,
            OrderId,
            OrderItemNumber,
            Product2Id,
            Product_Name__c,
            Product_Family__c,
            Quantity,
            UnitPrice,
            TotalPrice,
            ListPrice,
            Total_MRC_Amortized__c,
            NRC_IRU_FEE__c,
            NRC_Non_Amortized__c,
            Vendor_Fees_Monthly__c,
            Vendor_NRC__c,
            ServiceDate,
            EndDate,
            Contract_End_Month__c,
            Description,
            Bandwidth_NEW__c,
            Bandwidth_Numerical__c,
            Term__c,
            Product_Service_Term__c,
            Cancelled__c,
            Last_Mile_Carrier__c,
            Vendor_Last_Mile_CID__c,
            SBQQ__ChargeType__c,
            SBQQ__BillingFrequency__c,
            SBQQ__Status__c,
            OFF_NET_IDs__c,
            CreatedDate,
            LastModifiedDate
        FROM OrderItem
        WHERE OrderId IN ('{ids_str}')
        """

        result = es_sf.query_all(orderitem_query)
        all_order_items.extend(result["records"])

        if (i // chunk_size + 1) % 10 == 0:
            print(
                f"   Chunk {i//chunk_size + 1}: {len(all_order_items):,} items so far..."
            )

    order_items_df = pd.DataFrame(all_order_items)
    if len(order_items_df) > 0:
        order_items_df = order_items_df.drop(columns=["attributes"], errors="ignore")

    print(f"\n✅ Total OrderItems retrieved: {len(order_items_df):,}")

    # Show basic stats
    if len(order_items_df) > 0:
        items_per_order = order_items_df.groupby("OrderId").size()
        print(f"\n📊 OrderItem Statistics:")
        print(f"   Orders with items: {len(items_per_order):,}")
        print(f"   Min items per order: {items_per_order.min()}")
        print(f"   Max items per order: {items_per_order.max()}")
        print(f"   Avg items per order: {items_per_order.mean():.1f}")
else:
    order_items_df = pd.DataFrame()
    print("⚠️ No orders ready to migrate, skipping OrderItem query")


STEP 13B: QUERYING ORDER ITEMS

Querying OrderItems for 11,475 orders...
   Chunk 10: 2,565 items so far...
   Chunk 20: 5,539 items so far...
   Chunk 30: 7,942 items so far...
   Chunk 40: 10,003 items so far...
   Chunk 50: 13,169 items so far...
   Chunk 60: 16,267 items so far...
   Chunk 70: 18,628 items so far...

✅ Total OrderItems retrieved: 20,608

📊 OrderItem Statistics:
   Orders with items: 11,475
   Min items per order: 1
   Max items per order: 18
   Avg items per order: 1.8


In [20]:
# === STEP 13C: ORDER ITEM ANALYSIS ===

print("\n" + "=" * 80)
print("STEP 13C: ORDER ITEM ANALYSIS")
print("=" * 80)

orderitem_summary = []
orderitem_data_quality = []

if len(order_items_df) > 0:
    total_items = len(order_items_df)

    # === PRODUCT FAMILY BREAKDOWN ===
    print("\n📊 Product Family Breakdown:")
    family_counts = order_items_df["Product_Family__c"].value_counts(dropna=False)
    for family, count in family_counts.head(15).items():
        pct = 100 * count / total_items
        family_name = family if family else "(null)"
        print(f"   {family_name}: {count:,} ({pct:.1f}%)")
    if len(family_counts) > 15:
        print(f"   ... and {len(family_counts) - 15} more families")

    # === CHARGE TYPE BREAKDOWN ===
    print("\n📊 Charge Type Breakdown (SBQQ__ChargeType__c):")
    charge_counts = order_items_df["SBQQ__ChargeType__c"].value_counts(dropna=False)
    for charge, count in charge_counts.items():
        pct = 100 * count / total_items
        charge_name = charge if charge else "(null)"
        print(f"   {charge_name}: {count:,} ({pct:.1f}%)")

    # === CANCELLED BREAKDOWN ===
    print("\n📊 Cancelled Status:")
    cancelled_counts = order_items_df["Cancelled__c"].value_counts(dropna=False)
    for status, count in cancelled_counts.items():
        pct = 100 * count / total_items
        status_name = str(status) if status is not None else "(null)"
        print(f"   Cancelled={status_name}: {count:,} ({pct:.1f}%)")

    # === MRC/NRC ANALYSIS ===
    print("\n📊 Financial Fields:")

    # Total MRC
    mrc_populated = order_items_df["Total_MRC_Amortized__c"].notna().sum()
    mrc_sum = order_items_df["Total_MRC_Amortized__c"].sum()
    print(
        f"   Total_MRC_Amortized__c: {mrc_populated:,} populated, sum=${mrc_sum:,.2f}"
    )

    # NRC
    nrc_populated = order_items_df["NRC_IRU_FEE__c"].notna().sum()
    nrc_sum = order_items_df["NRC_IRU_FEE__c"].sum()
    print(f"   NRC_IRU_FEE__c: {nrc_populated:,} populated, sum=${nrc_sum:,.2f}")

    # Vendor costs
    vendor_mrc_populated = order_items_df["Vendor_Fees_Monthly__c"].notna().sum()
    vendor_mrc_sum = order_items_df["Vendor_Fees_Monthly__c"].sum()
    print(
        f"   Vendor_Fees_Monthly__c: {vendor_mrc_populated:,} populated, sum=${vendor_mrc_sum:,.2f}"
    )

    # === DATA QUALITY CHECKS ===
    print("\n📊 OrderItem Data Quality:")

    # Missing Product Name
    missing_product = order_items_df["Product_Name__c"].isna().sum()
    if missing_product > 0:
        orderitem_data_quality.append(
            {
                "Issue": "OrderItems missing Product_Name__c",
                "Count": missing_product,
                "Percentage": f"{100*missing_product/total_items:.1f}%",
                "Severity": "MEDIUM",
                "Impact": "Need product mapping for Service_Charge__c",
            }
        )
        print(
            f"   ⚠️ Missing Product_Name__c: {missing_product:,} ({100*missing_product/total_items:.1f}%)"
        )

    # Missing Product Family
    missing_family = order_items_df["Product_Family__c"].isna().sum()
    if missing_family > 0:
        orderitem_data_quality.append(
            {
                "Issue": "OrderItems missing Product_Family__c",
                "Count": missing_family,
                "Percentage": f"{100*missing_family/total_items:.1f}%",
                "Severity": "LOW",
                "Impact": "Can use Product_Name__c as fallback",
            }
        )
        print(
            f"   ⚠️ Missing Product_Family__c: {missing_family:,} ({100*missing_family/total_items:.1f}%)"
        )

    # Missing Unit Price
    missing_price = order_items_df["UnitPrice"].isna().sum()
    if missing_price > 0:
        orderitem_data_quality.append(
            {
                "Issue": "OrderItems missing UnitPrice",
                "Count": missing_price,
                "Percentage": f"{100*missing_price/total_items:.1f}%",
                "Severity": "MEDIUM",
                "Impact": "Need price for Service_Charge__c.Amount__c",
            }
        )
        print(
            f"   ⚠️ Missing UnitPrice: {missing_price:,} ({100*missing_price/total_items:.1f}%)"
        )

    # Cancelled items
    cancelled_items = order_items_df["Cancelled__c"].fillna(False).sum()
    if cancelled_items > 0:
        orderitem_data_quality.append(
            {
                "Issue": "OrderItems marked as Cancelled",
                "Count": int(cancelled_items),
                "Percentage": f"{100*cancelled_items/total_items:.1f}%",
                "Severity": "INFO",
                "Impact": "Consider excluding from migration",
            }
        )
        print(
            f"   ℹ️ Cancelled items: {int(cancelled_items):,} ({100*cancelled_items/total_items:.1f}%)"
        )

    # Missing ServiceDate
    missing_svc_date = order_items_df["ServiceDate"].isna().sum()
    if missing_svc_date > 0:
        orderitem_data_quality.append(
            {
                "Issue": "OrderItems missing ServiceDate",
                "Count": missing_svc_date,
                "Percentage": f"{100*missing_svc_date/total_items:.1f}%",
                "Severity": "LOW",
                "Impact": "Can use Order-level date or OSS bill_start_date",
            }
        )
        print(
            f"   ⚠️ Missing ServiceDate: {missing_svc_date:,} ({100*missing_svc_date/total_items:.1f}%)"
        )

    # === BUILD SUMMARY ===
    orderitem_summary = [
        {"Metric": "Total OrderItems", "Value": total_items},
        {"Metric": "Unique Orders", "Value": order_items_df["OrderId"].nunique()},
        {
            "Metric": "Unique Product Families",
            "Value": order_items_df["Product_Family__c"].nunique(),
        },
        {
            "Metric": "Unique Product Names",
            "Value": order_items_df["Product_Name__c"].nunique(),
        },
        {"Metric": "Total MRC (sum)", "Value": f"${mrc_sum:,.2f}"},
        {"Metric": "Total NRC (sum)", "Value": f"${nrc_sum:,.2f}"},
        {"Metric": "Total Vendor MRC (sum)", "Value": f"${vendor_mrc_sum:,.2f}"},
        {"Metric": "Cancelled Items", "Value": int(cancelled_items)},
        {
            "Metric": "Active Items (not cancelled)",
            "Value": total_items - int(cancelled_items),
        },
    ]

    print("\n✅ OrderItem analysis complete")
else:
    print("⚠️ No OrderItems to analyze")

orderitem_summary_df = pd.DataFrame(orderitem_summary)
orderitem_dq_df = pd.DataFrame(orderitem_data_quality)


STEP 13C: ORDER ITEM ANALYSIS

📊 Product Family Breakdown:
   Point-to-Point (PTPS): 4,986 (24.2%)
   Dedicated Internet Access (DIAS): 3,461 (16.8%)
   IP: 3,323 (16.1%)
   Hosted Voice (VOIC): 1,648 (8.0%)
   Dark Fiber (DFBR): 1,467 (7.1%)
   Point-to-MultiPoint (PMPS): 1,340 (6.5%)
   Promotions: 876 (4.3%)
   Managed Service (MSP): 771 (3.7%)
   Handoff Type: 575 (2.8%)
   Diversity: 497 (2.4%)
   Additional Port: 277 (1.3%)
   Tagged / Untagged: 234 (1.1%)
   Logical Attributes: 230 (1.1%)
   Routing: 125 (0.6%)
   Managed Wave (MWAV): 106 (0.5%)
   ... and 34 more families

📊 Charge Type Breakdown (SBQQ__ChargeType__c):
   (null): 20,608 (100.0%)

📊 Cancelled Status:
   Cancelled=False: 20,608 (100.0%)

📊 Financial Fields:
   Total_MRC_Amortized__c: 20,608 populated, sum=$9,650,224.07
   NRC_IRU_FEE__c: 8,376 populated, sum=$14,307,429.82
   Vendor_Fees_Monthly__c: 17 populated, sum=$8,780.00

📊 OrderItem Data Quality:

✅ OrderItem analysis complete


In [21]:
# === STEP 13D: PRODUCT FAMILY SUMMARY ===

print("\n" + "=" * 80)
print("STEP 13D: PRODUCT FAMILY SUMMARY")
print("=" * 80)

if len(order_items_df) > 0:
    # Create product family summary with financials
    product_summary = (
        order_items_df.groupby("Product_Family__c", dropna=False)
        .agg(
            {
                "Id": "count",
                "OrderId": "nunique",
                "Total_MRC_Amortized__c": "sum",
                "NRC_IRU_FEE__c": "sum",
                "Vendor_Fees_Monthly__c": "sum",
                "UnitPrice": "mean",
            }
        )
        .reset_index()
    )

    product_summary.columns = [
        "Product_Family",
        "Item_Count",
        "Order_Count",
        "Total_MRC",
        "Total_NRC",
        "Total_Vendor_MRC",
        "Avg_UnitPrice",
    ]

    # Sort by item count
    product_summary = product_summary.sort_values("Item_Count", ascending=False)

    # Fill nulls for display
    product_summary["Product_Family"] = product_summary["Product_Family"].fillna(
        "(null)"
    )

    print(f"\n✅ Product Family Summary created: {len(product_summary)} families")
    print("\nTop 10 by Item Count:")
    print(
        product_summary.head(10)[
            ["Product_Family", "Item_Count", "Order_Count", "Total_MRC"]
        ].to_string()
    )
else:
    product_summary = pd.DataFrame()

product_summary_df = product_summary


STEP 13D: PRODUCT FAMILY SUMMARY

✅ Product Family Summary created: 49 families

Top 10 by Item Count:
                      Product_Family  Item_Count  Order_Count   Total_MRC
33             Point-to-Point (PTPS)        4986         4969  3963154.84
11  Dedicated Internet Access (DIAS)        3461         3431  2932247.50
21                                IP        3323         3165     4473.25
20               Hosted Voice (VOIC)        1648          417   189779.34
6                  Dark Fiber (DFBR)        1467          771   601983.15
32        Point-to-MultiPoint (PMPS)        1340         1340   970607.99
37                        Promotions         876          868    -9700.00
26             Managed Service (MSP)         771          270   126825.24
18                      Handoff Type         575          573        0.00
13                         Diversity         497          247       20.00


In [22]:
# === STEP 14: CREATE BAN MAPPING TABLE ===

print("\n" + "=" * 80)
print("STEP 14: CREATING BAN MAPPING TABLE")
print("=" * 80)

if len(orders_ready_df) > 0:
    ban_mapping_data = []
    legacy_ban_counts = orders_ready_df.groupby("Billing_Invoice__c").size().to_dict()

    for legacy_id, bbf_ban in legacy_to_bbf_ban.items():
        order_count = legacy_ban_counts.get(legacy_id, 0)
        if order_count > 0:
            ban_mapping_data.append(
                {
                    "Legacy_BAN_Id": legacy_id,
                    "New_BBF_BAN_Id": bbf_ban["Id"],
                    "New_BBF_BAN_Name": bbf_ban["Name"],
                    "Account__c": bbf_ban["Account__c"],
                    "Account_Name": bbf_ban["Account_Name"],
                    "Order_Count": order_count,
                }
            )

    ban_mapping_df = pd.DataFrame(ban_mapping_data)
    ban_mapping_df = ban_mapping_df.sort_values("Order_Count", ascending=False)
    print(f"\n✅ BAN mappings with orders: {len(ban_mapping_df):,}")
else:
    ban_mapping_df = pd.DataFrame()


STEP 14: CREATING BAN MAPPING TABLE

✅ BAN mappings with orders: 2,440


In [23]:
# === STEP 15: GENERATE SUMMARY ===

print("\n" + "=" * 80)
print("STEP 15: MIGRATION SUMMARY")
print("=" * 80)

# Get OrderItem count safely
orderitem_count = (
    len(order_items_df) if "order_items_df" in dir() and len(order_items_df) > 0 else 0
)
product_family_count = (
    order_items_df["Product_Family__c"].nunique() if orderitem_count > 0 else 0
)

summary_data = [
    {"Category": "FILTER PIPELINE", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "1. Total Active Status Orders",
        "Count": len(all_orders_df),
        "Notes": "All orders with qualifying status",
    },
    {
        "Category": "",
        "Metric": "2. Excluded (PA MARKET DECOM)",
        "Count": len(excluded_pa_decom_df),
        "Notes": "Project_Group__c contains 'PA MARKET DECOM'",
    },
    {
        "Category": "",
        "Metric": "3. Excluded (Work Orders)",
        "Count": len(excluded_work_orders_df),
        "Notes": f"Service_Order_Record_Type__c != '{VALID_RECORD_TYPE}'",
    },
    {
        "Category": "",
        "Metric": "4. Excluded (Not Actively Billing)",
        "Count": len(excluded_not_billing_df),
        "Notes": f"OSS: not in {ACTIVE_OSS_STATES} or billing dates invalid",
    },
    {
        "Category": "",
        "Metric": "5. Confirmed Actively Billing",
        "Count": len(active_orders_df),
        "Notes": "Passed all filters",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "BAN MAPPING", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Ready (has BBF BAN)",
        "Count": len(orders_ready_df),
        "Notes": "Can be migrated now",
    },
    {
        "Category": "",
        "Metric": "Missing BBF BAN",
        "Count": len(orders_no_bbf_ban_df),
        "Notes": "Need new BBF BAN created",
    },
    {
        "Category": "",
        "Metric": "Missing ANY BAN",
        "Count": len(orders_no_ban_df),
        "Notes": "CRITICAL - no BAN reference",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "ORDER ITEMS (v4)", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Total OrderItems",
        "Count": orderitem_count,
        "Notes": "For orders ready to migrate",
    },
    {
        "Category": "",
        "Metric": "Unique Product Families",
        "Count": product_family_count,
        "Notes": "Distinct Product_Family__c values",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "RECORDS TO MIGRATE", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "BANs (with orders)",
        "Count": len(ban_mapping_df),
        "Notes": "Billing_Invoice__c to BAN__c",
    },
    {
        "Category": "",
        "Metric": "Accounts",
        "Count": len(accounts_df),
        "Notes": "Unique accounts from BBF BANs",
    },
    {
        "Category": "",
        "Metric": "Contacts",
        "Count": len(contacts_df),
        "Notes": "Contacts for migration accounts",
    },
    {
        "Category": "",
        "Metric": "Locations",
        "Count": len(locations_df),
        "Notes": "Address_A + Address_Z from orders",
    },
    {
        "Category": "",
        "Metric": "Off_Net",
        "Count": len(offnet_df),
        "Notes": "Off_Net for migration locations",
    },
]

summary_df = pd.DataFrame(summary_data)

print(
    f"""
📊 SUMMARY
{'─'*50}
FILTER PIPELINE:
  Total Active Status Orders:     {len(all_orders_df):,}
  ├─ Excluded (PA MARKET DECOM):  {len(excluded_pa_decom_df):,}
  ├─ Excluded (Work Orders):      {len(excluded_work_orders_df):,}
  ├─ Excluded (Not Billing OSS):  {len(excluded_not_billing_df):,}
  └─ Confirmed Actively Billing:  {len(active_orders_df):,}

BAN MAPPING:
  ├─ Ready to migrate:            {len(orders_ready_df):,}
  ├─ Missing BBF BAN:             {len(orders_no_bbf_ban_df):,}
  └─ Missing ANY BAN:             {len(orders_no_ban_df):,}

ORDER ITEMS (v4 NEW):
  ├─ Total OrderItems:            {orderitem_count:,}
  └─ Unique Product Families:     {product_family_count:,}

RECORDS TO MIGRATE:
  ├─ BANs:                        {len(ban_mapping_df):,}
  ├─ Accounts:                    {len(accounts_df):,}
  ├─ Contacts:                    {len(contacts_df):,}
  ├─ Locations:                   {len(locations_df):,}
  └─ Off_Net:                     {len(offnet_df):,}
"""
)


STEP 15: MIGRATION SUMMARY

📊 SUMMARY
──────────────────────────────────────────────────
FILTER PIPELINE:
  Total Active Status Orders:     17,972
  ├─ Excluded (PA MARKET DECOM):  887
  ├─ Excluded (Work Orders):      3,477
  ├─ Excluded (Not Billing OSS):  2,119
  └─ Confirmed Actively Billing:  11,489

BAN MAPPING:
  ├─ Ready to migrate:            11,475
  ├─ Missing BBF BAN:             12
  └─ Missing ANY BAN:             2

ORDER ITEMS (v4 NEW):
  ├─ Total OrderItems:            20,608
  └─ Unique Product Families:     49

RECORDS TO MIGRATE:
  ├─ BANs:                        2,440
  ├─ Accounts:                    2,224
  ├─ Contacts:                    15,576
  ├─ Locations:                   10,175
  └─ Off_Net:                     2,157



In [24]:
# === STEP 16: EXPORT TO EXCEL ===

print("\n" + "=" * 80)
print("STEP 16: EXPORTING TO EXCEL")
print("=" * 80)

wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)


def write_df_to_sheet(ws, df, start_row=1):
    """Write dataframe to worksheet with formatting"""
    for r_idx, row in enumerate(
        dataframe_to_rows(df, index=False, header=True), start=start_row
    ):
        for c_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            if r_idx == start_row:
                cell.font = header_font
                cell.fill = header_fill
            cell.border = thin_border


# --- SHEET 1: Summary ---
ws_summary.append(["ES → BBF Migration Analysis (v4)"])
ws_summary["A1"].font = Font(bold=True, size=16)
ws_summary.append([f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"])
ws_summary.append(
    [
        f"OSS Active States: {ACTIVE_OSS_STATES} | Record Type filter | OSS Billing validation | OrderItem Analysis"
    ]
)
ws_summary.append([])
write_df_to_sheet(ws_summary, summary_df, start_row=5)

ws_summary.column_dimensions["A"].width = 20
ws_summary.column_dimensions["B"].width = 40
ws_summary.column_dimensions["C"].width = 12
ws_summary.column_dimensions["D"].width = 55
print("   ✅ Summary")

# --- SHEET 2: Active Orders (Ready) ---
if len(orders_ready_df) > 0:
    ws = wb.create_sheet("Active_Orders")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "Billing_Invoice__c",
        "New_BBF_BAN_Id",
        "New_BBF_BAN_Name",
        "Address_A__c",
        "Address_Z__c",
        "SOF_MRC__c",
        "Service_Start_Date__c",
        "Billing_Start_Date__c",
        "OSS_Order__c",
        "order_state_cd",
        "bill_start_date",
        "bill_end_date",
    ]
    export_cols = [c for c in export_cols if c in orders_ready_df.columns]
    write_df_to_sheet(ws, orders_ready_df[export_cols])
    print(f"   ✅ Active_Orders ({len(orders_ready_df):,} records)")

# --- SHEET 3: Order Items (NEW in v4) ---
if "order_items_df" in dir() and len(order_items_df) > 0:
    ws = wb.create_sheet("OrderItems")
    export_cols = [
        "Id",
        "OrderId",
        "OrderItemNumber",
        "Product_Name__c",
        "Product_Family__c",
        "Quantity",
        "UnitPrice",
        "TotalPrice",
        "Total_MRC_Amortized__c",
        "NRC_IRU_FEE__c",
        "Vendor_Fees_Monthly__c",
        "ServiceDate",
        "EndDate",
        "Bandwidth_NEW__c",
        "Term__c",
        "Cancelled__c",
        "SBQQ__ChargeType__c",
        "Last_Mile_Carrier__c",
        "OFF_NET_IDs__c",
    ]
    export_cols = [c for c in export_cols if c in order_items_df.columns]
    write_df_to_sheet(ws, order_items_df[export_cols])
    print(f"   ✅ OrderItems ({len(order_items_df):,} records)")

# --- SHEET 4: OrderItem Summary (NEW in v4) ---
if "orderitem_summary_df" in dir() and len(orderitem_summary_df) > 0:
    ws = wb.create_sheet("OrderItem_Summary")
    write_df_to_sheet(ws, orderitem_summary_df)
    print(f"   ✅ OrderItem_Summary ({len(orderitem_summary_df):,} metrics)")

# --- SHEET 5: Product Family Summary (NEW in v4) ---
if "product_summary_df" in dir() and len(product_summary_df) > 0:
    ws = wb.create_sheet("Product_Family_Summary")
    write_df_to_sheet(ws, product_summary_df)
    print(f"   ✅ Product_Family_Summary ({len(product_summary_df):,} families)")

# --- SHEET 6: BAN Mapping ---
if len(ban_mapping_df) > 0:
    ws = wb.create_sheet("BAN_Mapping")
    write_df_to_sheet(ws, ban_mapping_df)
    print(f"   ✅ BAN_Mapping ({len(ban_mapping_df):,} records)")

# --- SHEET 7: Accounts ---
if len(accounts_df) > 0:
    ws = wb.create_sheet("Accounts")
    write_df_to_sheet(ws, accounts_df)
    print(f"   ✅ Accounts ({len(accounts_df):,} records)")

# --- SHEET 8: Contacts ---
if len(contacts_df) > 0:
    ws = wb.create_sheet("Contacts")
    write_df_to_sheet(ws, contacts_df)
    print(f"   ✅ Contacts ({len(contacts_df):,} records)")

# --- SHEET 9: Locations ---
if len(locations_df) > 0:
    ws = wb.create_sheet("Locations")
    write_df_to_sheet(ws, locations_df)
    print(f"   ✅ Locations ({len(locations_df):,} records)")

# --- SHEET 10: Off_Net ---
if len(offnet_df) > 0:
    ws = wb.create_sheet("Off_Net")
    write_df_to_sheet(ws, offnet_df)
    print(f"   ✅ Off_Net ({len(offnet_df):,} records)")

# --- SHEET 11: Data Quality (combined Order + OrderItem) ---
combined_dq = data_quality_issues.copy()
if "orderitem_data_quality" in dir() and len(orderitem_data_quality) > 0:
    combined_dq.extend(orderitem_data_quality)

if len(combined_dq) > 0:
    ws = wb.create_sheet("Data_Quality")
    dq_df = pd.DataFrame(combined_dq)
    write_df_to_sheet(ws, dq_df)
    print(f"   ✅ Data_Quality ({len(combined_dq):,} issues)")

# --- SHEET 12: Orders Missing BBF BAN ---
if len(orders_no_bbf_ban_df) > 0:
    ws = wb.create_sheet("Orders_Missing_BBF_BAN")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "Billing_Invoice__c",
        "SOF_MRC__c",
    ]
    export_cols = [c for c in export_cols if c in orders_no_bbf_ban_df.columns]
    write_df_to_sheet(ws, orders_no_bbf_ban_df[export_cols])
    print(f"   ✅ Orders_Missing_BBF_BAN ({len(orders_no_bbf_ban_df):,} records)")

# --- SHEET 13: Excluded - PA MARKET DECOM ---
if len(excluded_pa_decom_df) > 0:
    ws = wb.create_sheet("Excluded_PA_DECOM")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Project_Group__c",
        "Account_Name",
    ]
    export_cols = [c for c in export_cols if c in excluded_pa_decom_df.columns]
    write_df_to_sheet(ws, excluded_pa_decom_df[export_cols])
    print(f"   ✅ Excluded_PA_DECOM ({len(excluded_pa_decom_df):,} records)")

# --- SHEET 14: Excluded - Work Orders (enriched with OSS data) ---
if len(excluded_work_orders_df) > 0:
    ws = wb.create_sheet("Excluded_Work_Orders")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Service_Order_Record_Type__c",
        "Account_Name",
        "OSS_Order__c",
        "order_nm",
        "order_id",
        "workorder_type_cd",
        "workorder_type_desc",
        "workorder_state_cd",
        "workorder_state_desc",
        "description",
        "start_date",
        "end_date",
    ]
    export_cols = [c for c in export_cols if c in excluded_work_orders_df.columns]
    write_df_to_sheet(ws, excluded_work_orders_df[export_cols])
    print(f"   ✅ Excluded_Work_Orders ({len(excluded_work_orders_df):,} records)")

# --- SHEET 15: Excluded - Not Actively Billing ---
if len(excluded_not_billing_df) > 0:
    ws = wb.create_sheet("Excluded_Not_Billing")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "OSS_Order__c",
        "order_state_cd",
        "OSS_State_Desc",
        "bill_start_date",
        "bill_end_date",
        "OSS_Billing_Status",
    ]
    export_cols = [c for c in export_cols if c in excluded_not_billing_df.columns]
    write_df_to_sheet(ws, excluded_not_billing_df[export_cols])
    print(f"   ✅ Excluded_Not_Billing ({len(excluded_not_billing_df):,} records)")

# --- SHEET 16: OrderItem Data Quality (NEW in v4) ---
if "orderitem_dq_df" in dir() and len(orderitem_dq_df) > 0:
    ws = wb.create_sheet("OrderItem_Data_Quality")
    write_df_to_sheet(ws, orderitem_dq_df)
    print(f"   ✅ OrderItem_Data_Quality ({len(orderitem_dq_df):,} issues)")

# Save
wb.save(OUTPUT_FILE)
print(f"\n✅ Excel file saved: {OUTPUT_FILE}")


STEP 16: EXPORTING TO EXCEL
   ✅ Summary
   ✅ Active_Orders (11,475 records)
   ✅ OrderItems (20,608 records)
   ✅ OrderItem_Summary (9 metrics)
   ✅ Product_Family_Summary (49 families)
   ✅ BAN_Mapping (2,440 records)
   ✅ Accounts (2,224 records)
   ✅ Contacts (15,576 records)
   ✅ Locations (10,175 records)
   ✅ Off_Net (2,157 records)
   ✅ Data_Quality (3 issues)
   ✅ Orders_Missing_BBF_BAN (12 records)
   ✅ Excluded_PA_DECOM (887 records)
   ✅ Excluded_Work_Orders (3,477 records)
   ✅ Excluded_Not_Billing (2,119 records)

✅ Excel file saved: es_bbf_migration_analysis_v4_20260107_212249.xlsx


In [25]:
# === CLEANUP ===

print("\n" + "=" * 80)
print("CLEANUP")
print("=" * 80)

oss_conn.close()
print("✅ OSS connection closed")

print(f"\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print(f"\nOutput file: {OUTPUT_FILE}")


CLEANUP
✅ OSS connection closed

ANALYSIS COMPLETE

Output file: es_bbf_migration_analysis_v4_20260107_212249.xlsx


---
## v4 Changes Summary

### NEW: OrderItem Analysis
- Queries all OrderItems for orders in migration scope
- Product Family breakdown with financial totals
- Data quality checks on OrderItems
- New sheets: OrderItems, OrderItem_Summary, Product_Family_Summary, OrderItem_Data_Quality

### Inherited from v3
- OSS Active States: CL (Closed) and OA (Accepted)
- OSS `bill_start_date` available when SF `Service_Start_Date__c` is missing
- Work Orders enriched with OSS workorder data

### Field Mapping Reference
See companion document: `ES_BBF_Field_Mapping_Proposal.md` for:
- ES Order → BBF Service__c mapping
- ES OrderItem → BBF Service_Charge__c mapping
- Status value mappings
- Product/Charge type mappings

### Next Steps
1. Review field mapping proposal
2. Create Product mapping table (ES Product_Family → BBF Product_Simple)
3. Create Status mapping table (ES Status → BBF Status__c)
4. Create Service__c migration notebook
5. Create Service_Charge__c migration notebook